## Import datasets

In [77]:
from pandas import read_csv

print("--------------------")
print("X RANDOM vs O RANDOM:")
rr = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7921041/random_random_1000.csv")
rr["dataset_id"] = "random_vs_random"
print(rr["outcome"].value_counts(normalize=True))

print("--------------------")
print("X RANDOM vs O EXPERT:")
rm = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7921043/random_minimaxab_1000.csv")
rm["dataset_id"] = "random_vs_expert"
print(rm["outcome"].value_counts(normalize=True))

print("--------------------")
print("X EXPERT vs O RANDOM:")
mr = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7921050/minimaxab_random_1000.csv")
mr["dataset_id"] = "expert_vs_random"
print(mr["outcome"].value_counts(normalize=True))

print("--------------------")
print("X EXPERT vs O EXPERT:")
mm = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7921045/minimaxab_minimaxab_1000.csv")
print(mm["outcome"].value_counts(normalize=True))
mm["dataset_id"] = "expert_vs_expert"


--------------------
X RANDOM vs O RANDOM:
1.0    0.460388
0.0    0.386149
0.5    0.153463
Name: outcome, dtype: float64
--------------------
X RANDOM vs O EXPERT:
1.0    0.381239
0.0    0.381239
0.5    0.237522
Name: outcome, dtype: float64
--------------------
X EXPERT vs O RANDOM:
1.0    0.568935
0.0    0.398080
0.5    0.032984
Name: outcome, dtype: float64
--------------------
X EXPERT vs O EXPERT:
0.5    1.0
Name: outcome, dtype: float64


## Combine Datasets

In [78]:
from pandas import concat

combined_df = concat([rr, rm, mr, mm])

print(combined_df)

print("-------------")
print("OUTCOMES:")
print(combined_df["outcome"].value_counts(normalize=True))

print("-------------")
print("DATASETS:")
print(combined_df["dataset_id"].value_counts(normalize=True))

      game_id  move_id a1 b1 c1 a2  ... a3 b3 c3 player outcome        dataset_id
0           1        1  X  -  -  -  ...  -  -  -      X     1.0  random_vs_random
1           1        2  X  -  -  -  ...  -  O  -      O     0.0  random_vs_random
2           1        3  X  X  -  -  ...  -  O  -      X     1.0  random_vs_random
3           1        4  X  X  -  -  ...  -  O  -      O     0.0  random_vs_random
4           1        5  X  X  -  X  ...  -  O  -      X     1.0  random_vs_random
...       ...      ... .. .. .. ..  ... .. .. ..    ...     ...               ...
8995     1000        5  X  X  O  -  ...  X  -  -      X     0.5  expert_vs_expert
8996     1000        6  X  X  O  O  ...  X  -  -      O     0.5  expert_vs_expert
8997     1000        7  X  X  O  O  ...  X  -  -      X     0.5  expert_vs_expert
8998     1000        8  X  X  O  O  ...  X  O  -      O     0.5  expert_vs_expert
8999     1000        9  X  X  O  O  ...  X  O  X      X     0.5  expert_vs_expert

[29326 rows x 1

In [79]:
combined_df.to_csv("move_values.csv", index=False)


# Separate Player Perspectives

In [80]:
df = combined_df.copy()

x_rows = df[df["player"] == "X"]
o_rows = df[df["player"] == "O"]
print("X ROWS:", len(x_rows))
print("O ROWS:", len(o_rows))
print("TOTAL :", len(df))

X ROWS: 16103
O ROWS: 13223
TOTAL : 29326


In [81]:
o_rows.to_csv("move_values_o.csv")
x_rows.to_csv("move_values_x.csv")

## Normalize O Player Perspective

In [82]:
OPPOSITE_PERSPECTIVES = {"X":"O", "O":"X", "-":"-"}
OPPOSITE_OUTCOMES = {1:0, 0:1, 0.5:0.5}

def normalize_square(val):
    return OPPOSITE_PERSPECTIVES[val]

#def normalize_board(board_notation):
#    return "".join([normalize(char) for char in board_notation])

def normalize_outcome(val):
    return OPPOSITE_OUTCOMES[val]

assert normalize("X") == "O"
assert normalize("O") == "X"
assert normalize("-") == "-"
#assert normalize_board("XO-XO----") == "OX-OX----"

assert normalize_outcome(1) == 0
assert normalize_outcome(0.5) == 0.5
assert normalize_outcome(0) == 1


In [83]:
print("\n--------------------")
print("ORIGINAL PERSPECTIVES:")
print(o_rows[["a1","b1", "c1", "a2","b2", "c2", "a3","b3", "c3", "player", "outcome"]].head())
print(o_rows["outcome"].value_counts(normalize=True))

# NORMALIZING...
new_o_rows = o_rows.copy()

#new_o_rows["board_state"] = o_rows["board_state"].apply(normalize_board)
for col in ["a1","b1","c1","a2","b2","c2","a3","b3","c3"]:
    new_o_rows[col] = o_rows[col].apply(normalize)
new_o_rows["outcome"] = o_rows["outcome"].apply(normalize_outcome)
new_o_rows["player"] = "X" # normalize player (or drop it)

print("\n--------------------")
print("NORMALIZED PERSPECTIVES:")
print(new_o_rows[["a1","b1", "c1", "a2","b2", "c2", "a3","b3", "c3", "player", "outcome"]].head())
print(new_o_rows["outcome"].value_counts(normalize=True))


--------------------
ORIGINAL PERSPECTIVES:
   a1 b1 c1 a2 b2 c2 a3 b3 c3 player  outcome
1   X  -  -  -  -  -  -  O  -      O      0.0
3   X  X  -  -  O  -  -  O  -      O      0.0
5   X  X  -  X  O  -  -  O  O      O      0.0
8   -  -  -  O  -  -  -  X  -      O      0.0
10  X  -  -  O  -  O  -  X  -      O      0.0
0.5    0.403842
0.0    0.310141
1.0    0.286017
Name: outcome, dtype: float64

--------------------
NORMALIZED PERSPECTIVES:
   a1 b1 c1 a2 b2 c2 a3 b3 c3 player  outcome
1   O  -  -  -  -  -  -  X  -      X      1.0
3   O  O  -  -  X  -  -  X  -      X      1.0
5   O  O  -  O  X  -  -  X  X      X      1.0
8   -  -  -  X  -  -  -  O  -      X      1.0
10  O  -  -  X  -  X  -  O  -      X      1.0
0.5    0.403842
1.0    0.310141
0.0    0.286017
Name: outcome, dtype: float64


In [84]:
# https://pandas.pydata.org/docs/reference/api/pandas.concat.html
from pandas import concat

# single-player df
print("SINGLE PLAYER'S PERSPECTIVE:")
df_normalized = concat([x_rows, new_o_rows])
df_normalized.drop(columns=["player"], inplace=True)
df_normalized.sort_values(by=["dataset_id", "game_id", "move_id"], inplace=True)
print(len(df_normalized))

df_normalized

SINGLE PLAYER'S PERSPECTIVE:
29326


,game_id,move_id,a1,b1,c1,a2,b2,c2,a3,b3,c3,outcome,dataset_id
0,1,1,-,-,-,-,-,X,-,-,-,0.5,expert_vs_expert
1,1,2,-,-,X,-,-,O,-,-,-,0.5,expert_vs_expert
2,1,3,X,-,O,-,-,X,-,-,-,0.5,expert_vs_expert
3,1,4,O,-,X,X,-,O,-,-,-,0.5,expert_vs_expert
4,1,5,X,X,O,O,-,X,-,-,-,0.5,expert_vs_expert
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619,1000,1,-,X,-,-,-,-,-,-,-,1.0,random_vs_random
7620,1000,2,-,O,-,-,-,X,-,-,-,1.0,random_vs_random
7621,1000,3,-,X,X,-,-,O,-,-,-,1.0,random_vs_random
7622,1000,4,-,O,O,-,-,X,-,-,X,1.0,random_vs_random


In [85]:
df_normalized.to_csv("move_values_normalized.csv")